### Librerias

In [1]:
import pandas as pd
import requests
import io

### Repositario de los datos

In [2]:
periodos = "https://raw.githubusercontent.com/Federico-Higuera/FedericoHiguera/main/CuboOLAP/Periodos.csv"

In [3]:
aeropuertos = 'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/aeropuertos.csv'

In [4]:
aeropuertos_cambios = "https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/aeropuertos_cambios_infraestructura.csv"

In [5]:
#  2da etapa
vuelos = "https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/vuelosEtapa2.csv"

# Proceso ETL

### 1. Extracción de datos

In [6]:
df_vuelos = pd.read_csv(vuelos)

In [7]:
df_aeropuertos = pd.read_csv(aeropuertos)

In [8]:
df_cambios_aeropuertos = pd.read_csv(aeropuertos_cambios)

In [9]:
df_periodos = pd.read_csv(periodos)

### ............................................................................................................................
### Anuncios del negocio

Los expertos de Infraestructura visible luego de revisar los comentarios que les enviamos reportados por ustedes en la tarea de perfilamiento revisaron y efectivamente nos confirman los siguientes problemas de calidad en los datos
 
1.	Las fechas de construcción de aeropuertos no podían ser mayores a 2018    
2.	Había 79 aeropuertos duplicados, con fechas de construcción y vigencia diferentes
3.	El número de vuelos de cada aeropuerto tenía valores demasiado grandes
4.	Vuelos
1.	Había 34.718 vuelos duplicados
2.	16.946 duplicados con cambios en carga a bordo, carga ofrecida (eran el mismo vuelo pero con cargas distintas, lo cual no tenía sentido)
3.	Había aproximadamente 1186 vuelos con origen y destino igual, no tiene sentido que un vuelo salga por ejemplo de Medellín y como destino tenga Medellín
4.	Aproximadamente 51k de vuelos con valores nulos (nan) en las columnas trafico, vuelos, sillas y pasajeros, estos valores por columna equivalían a mínimo 1% y máximo 15% de los datos.
 
A partir de estas conclusiones, Infraestructura visible revisó las fuentes de datos entregadas, encontró los errores y les entrega un nuevo conjunto de datos que aseguran ya no tiene estos problemas. Así mismo, expande las fechas de los datos a registros de aeropuertos y vuelos hasta 2016 para ubicarnos en el tiempo en diciembre de 2016, incluye una descripción de las fuentes compartidas y el proceso de generación de las fuentes de cobertura y matriz de distancia y espera entregar el diccionario de datos cobertura.

## Perfilamiento de los datos
### VUELOS

### * Había 34.718 vuelos duplicados

In [10]:
df_vuelos.drop_duplicates().shape


(82559, 13)

In [11]:
print("Hay un total de ",df_vuelos.shape[0] - df_vuelos.drop_duplicates().shape[0], "registros duplicados en los datos de Vuelos")

Hay un total de  0 registros duplicados en los datos de Vuelos


### * 16.946 duplicados con cambios en carga a bordo, carga ofrecida (eran el mismo vuelo pero con cargas distintas, lo cual no tenía sentido)

In [12]:
# Se eliminan registros duplicados con diferentes valores en carga a bordo y carga ofrecida
df_vuelos.shape

(82559, 13)

In [13]:
df_vuelos.columns.tolist()

['ano',
 'mes',
 'origen',
 'destino',
 'tipo_equipo',
 'tipo_vuelo',
 'trafico',
 'empresa',
 'vuelos',
 'sillas',
 'carga_ofrecida',
 'pasajeros',
 'carga_bordo']

In [14]:
df_vuelos.drop_duplicates(inplace = True, subset = df_vuelos.columns.tolist()[0:8])

In [15]:
df_vuelos.shape

(82559, 13)

### * Había aproximadamente 1186 vuelos con origen y destino igual, no tiene sentido que un vuelo salga por ejemplo de Medellín y como destino tenga Medellín

In [16]:
# Vuelos con el mismo origen y destino
df_vuelos

,ano,mes,origen,destino,tipo_equipo,tipo_vuelo,trafico,empresa,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo
0,2010,3,ADZ,BOG,C560,T,N,CENTRAL CHARTER,1,0,0.0,7,0.0
1,2010,5,ADZ,BOG,C560,T,N,CENTRAL CHARTER,1,0,0.0,8,0.0
2,2010,6,ADZ,CLO,C560,T,N,CENTRAL CHARTER,1,0,0.0,8,0.0
3,2010,8,ADZ,CLO,C560,T,N,CENTRAL CHARTER,1,0,0.0,6,0.0
4,2010,10,ADZ,CLO,C560,T,N,CENTRAL CHARTER,1,0,0.0,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82554,2016,12,MVP,VVC,DC3,R,N,ALIANSA,1,19,1400.0,11,400.0
82555,2016,12,MVP,VVC,DC3T,R,N,ALIANSA,7,133,23800.0,100,1600.0
82556,2016,12,VVC,EJA,DC3T,R,N,ALIANSA,1,0,2150.0,0,2150.0
82557,2016,12,VVC,MVP,DC3,R,N,ALIANSA,1,10,1640.0,10,1640.0


In [17]:
filtro = df_vuelos['origen'] != df_vuelos['destino']

In [18]:
print( "Se eliminarán",len(filtro) - sum(filtro), "registros con el mismo origen y destino")

Se eliminarán 173 registros con el mismo origen y destino


In [19]:
df_vuelos = df_vuelos.where(filtro).dropna(how = 'all')

In [20]:
df_vuelos.shape

(82386, 13)

### * Aproximadamente 51k de vuelos con valores nulos (nan) en las columnas trafico, vuelos, sillas y pasajeros, estos valores por columna equivalían a mínimo 1% y máximo 15% de los datos

In [21]:
# vuelos con valores nulos (nan) en las columnas trafico, vuelos, sillas y pasajeros
valores_nulos = df_vuelos.shape[0] - df_vuelos[['trafico',"trafico", "vuelos", "sillas", "pasajeros"]].dropna(how = "any").shape[0]

In [22]:
print("existen", valores_nulos, "filas con valores 'nan' en las columnas trafico, vuelos, sillas y pasajeros")

existen 0 filas con valores 'nan' en las columnas trafico, vuelos, sillas y pasajeros


In [23]:
#Convertimos año y mes a enteros
df_vuelos.ano = df_vuelos.ano.astype(int)
df_vuelos.mes = df_vuelos.mes.astype(int)

In [24]:
#Seleccionamos las variables de interes para la tabla de hechos
df_vuelos = df_vuelos[['ano', 'mes', 'origen', 'destino', 'tipo_vuelo', 'vuelos', 'sillas', 'carga_ofrecida', 'pasajeros','carga_bordo','trafico']]

In [25]:
df_vuelos

,ano,mes,origen,destino,tipo_vuelo,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,trafico
0,2010,3,ADZ,BOG,T,1.0,0.0,0.0,7.0,0.0,N
1,2010,5,ADZ,BOG,T,1.0,0.0,0.0,8.0,0.0,N
2,2010,6,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N
3,2010,8,ADZ,CLO,T,1.0,0.0,0.0,6.0,0.0,N
4,2010,10,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...
82554,2016,12,MVP,VVC,R,1.0,19.0,1400.0,11.0,400.0,N
82555,2016,12,MVP,VVC,R,7.0,133.0,23800.0,100.0,1600.0,N
82556,2016,12,VVC,EJA,R,1.0,0.0,2150.0,0.0,2150.0,N
82557,2016,12,VVC,MVP,R,1.0,10.0,1640.0,10.0,1640.0,N


In [26]:
#Se asigna el atributo 'periodo' a la tabla de hechos de 'vuelo' 
df_vuelos['periodo'] = list(zip(df_vuelos.ano, df_vuelos.mes))

In [27]:
df_vuelos

,ano,mes,origen,destino,tipo_vuelo,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,trafico,periodo
0,2010,3,ADZ,BOG,T,1.0,0.0,0.0,7.0,0.0,N,"(2010, 3)"
1,2010,5,ADZ,BOG,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 5)"
2,2010,6,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 6)"
3,2010,8,ADZ,CLO,T,1.0,0.0,0.0,6.0,0.0,N,"(2010, 8)"
4,2010,10,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 10)"
...,...,...,...,...,...,...,...,...,...,...,...,...
82554,2016,12,MVP,VVC,R,1.0,19.0,1400.0,11.0,400.0,N,"(2016, 12)"
82555,2016,12,MVP,VVC,R,7.0,133.0,23800.0,100.0,1600.0,N,"(2016, 12)"
82556,2016,12,VVC,EJA,R,1.0,0.0,2150.0,0.0,2150.0,N,"(2016, 12)"
82557,2016,12,VVC,MVP,R,1.0,10.0,1640.0,10.0,1640.0,N,"(2016, 12)"


In [28]:
#Se asigna la columna periodo a la dimension "Periodos"
df_periodos['Periodo'] = list(zip(df_periodos.Ano, df_periodos.Mes))

In [29]:
df_periodos

,Id_Periodo,Periodo,Ano,Mes,Nombre_Mes,Q,Dias
0,0,"(2010, 1)",2010,1,Enero,Q1,31
1,1,"(2010, 2)",2010,2,Febrero,Q1,28
2,2,"(2010, 3)",2010,3,Marzo,Q1,31
3,3,"(2010, 4)",2010,4,Abril,Q2,30
4,4,"(2010, 5)",2010,5,Mayo,Q2,31
...,...,...,...,...,...,...,...
79,79,"(2016, 8)",2016,8,Agosto,Q3,31
80,80,"(2016, 9)",2016,9,Septiembre,Q3,30
81,81,"(2016, 10)",2016,10,Octubre,Q4,31
82,82,"(2016, 11)",2016,11,Noviembre,Q4,30


### AEROPUERTOS
### * Había 79 aeropuertos duplicados, con fechas de construcción y vigencia diferentes

In [30]:
df_aeropuertos.shape

(292, 23)

In [31]:
df_aeropuertos.shape[0] - df_aeropuertos.drop_duplicates(subset = ['sigla']).shape[0]

80

In [32]:
# Existen 80 aeropuertos duplicados, se eliminan los valores repetidos en la columna 'sigla'
df_aeropuertos.drop_duplicates(subset = ['sigla'], inplace = True)

In [33]:
df_aeropuertos.shape

(212, 23)

In [34]:
df_aeropuertos.head(1)

,Unnamed: 0,sigla,iata,nombre,municipio,departamento,categoria,latitud,longitud,propietario,...,pbmo,elevacion,resolucion,fecha_construccion,fecha_vigencia,clase,tipo,numero_vuelos_origen,gcd_departamento,gcd_municipio
0,274,BOG,BOG,EL DORADO,"Bogotá, D.C.","Bogotá, D.C.",Internacional,4.7017,-74.1469,AEROCIVIL,...,NaN,8356.0,"320,000",1999-12-09,NaN,4E,Aerocivil,1804153.0,11,11001


In [35]:
df_aeropuertos.columns.tolist()

['Unnamed: 0',
 'sigla',
 'iata',
 'nombre',
 'municipio',
 'departamento',
 'categoria',
 'latitud',
 'longitud',
 'propietario',
 'explotador',
 'longitud_pista',
 'ancho_pista',
 'pbmo',
 'elevacion',
 'resolucion',
 'fecha_construccion',
 'fecha_vigencia',
 'clase',
 'tipo',
 'numero_vuelos_origen',
 'gcd_departamento',
 'gcd_municipio']

### * Las fechas de construcción de aeropuertos no podían ser mayores a 2013

In [36]:
#Se eliminan aeropuertos que fueron construidos después de 2013
df_aeropuertos['Ano'] = df_aeropuertos['fecha_construccion'].str.split("-", expand = True)[0].astype(int)

In [37]:
df_aeropuertos.head(1)

,Unnamed: 0,sigla,iata,nombre,municipio,departamento,categoria,latitud,longitud,propietario,...,elevacion,resolucion,fecha_construccion,fecha_vigencia,clase,tipo,numero_vuelos_origen,gcd_departamento,gcd_municipio,Ano
0,274,BOG,BOG,EL DORADO,"Bogotá, D.C.","Bogotá, D.C.",Internacional,4.7017,-74.1469,AEROCIVIL,...,8356.0,"320,000",1999-12-09,NaN,4E,Aerocivil,1804153.0,11,11001,1999


In [38]:
filtro = df_aeropuertos['Ano'] <= 2013 #OJO: REVISAR

In [39]:
df_aeropuertos = df_aeropuertos.where(filtro).dropna(how = 'all')

In [40]:
df_cambios_aeropuertos.columns

Index(['sigla', 'iata', 'nombre', 'municipio', 'departamento', 'categoria',
       'latitud', 'longitud', 'propietario', 'explotador', 'longitud_pista',
       'ancho_pista', 'pbmo', 'elevacion', 'resolucion', 'fecha_construccion',
       'fecha_vigencia', 'clase', 'tipo', 'numero_vuelos_origen',
       'gcd_departamento', 'gcd_municipio', 'Ano'],
      dtype='object')

In [41]:
#Seleccionamos las variables de interes en los data frames
df_aeropuertos = df_aeropuertos[['sigla', 'nombre', 'municipio', 'departamento','categoria', 'latitud','longitud','fecha_construccion','clase', 'tipo','longitud_pista', 'ancho_pista', 'elevacion', 'pbmo','Ano']]
df_cambios_aeropuertos = df_cambios_aeropuertos[['sigla', 'nombre', 'municipio', 'departamento','categoria', 'latitud','longitud','fecha_construccion','clase', 'tipo','longitud_pista', 'ancho_pista', 'elevacion', 'pbmo','Ano']]

In [42]:
df_aeropuertos.shape
#df_cambios_aeropuertos.shape

(162, 15)

### * El número de vuelos de cada aeropuerto tenía valores demasiado grandes

In [43]:
numero_grande = 1500000

In [44]:
#filtro = df_aeropuertos.numero_vuelos_origen > numero_grande

In [45]:
#df_aeropuertos.where(filtro).dropna(how = 'all')

In [46]:
# Backups de Dataframes
#bu_vuelos = df_vuelos.copy()
#bu_aeropuertos = df_aeropuertos.copy()
#bu_periodos = df_periodos.copy()

### ............................................................................................................................
### 2. Transformación

In [47]:
df_aeropuertos.shape[1]
#df_cambios_aeropuertos.shape[1]

15

In [48]:
df_aeropuertos.head(5)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,pbmo,Ano
0,BOG,EL DORADO,"Bogotá, D.C.","Bogotá, D.C.",Internacional,4.7017,-74.1469,1999-12-09,4E,Aerocivil,3800.0,60.0,8356.0,NaN,1999.0
2,TLU,TOLU,Tolú Viejo,Sucre,Regional,9.5095,-75.5859,2000-12-24,2A,Aerocivil,1350.0,16.0,15.0,12202.0,2000.0
4,CUC,CAMILO DAZA,San José de Cúcuta,Norte de Santander,Internacional,7.9274,-72.5116,2013-04-28,4D,Aerocivil,1920.0,45.0,1027.0,NaN,2013.0
9,9MO,SAN NICOLAS,Tauramena,Casanare,Aeródromo,4.4703,-72.4575,2012-08-22,1A,Fumigación,840.0,10.0,585.0,2000.0,2012.0
10,TCO,LA FLORIDA,San Andrés de Tumaco,Nariño,Nacional,1.8144,-78.7490,1973-11-19,3C,Aerocivil,1600.0,30.0,10.0,NaN,1973.0


In [49]:
#Asignamos al DF aeropuertos la fecha inicial 
df_aeropuertos['fecha_ini'] = df_aeropuertos['fecha_construccion']

In [50]:
#Asignamos al DF aeropuertos cambios la fecha inicial
df_cambios_aeropuertos['fecha_ini'] = df_cambios_aeropuertos['Ano'].astype('str') +'-01-01'

In [51]:
#Obtenemos la dimension "aeropuertos historia"
df_aeropuertos_histo = df_aeropuertos.append(df_cambios_aeropuertos)
df_aeropuertos.shape

(162, 16)

In [52]:
df_aeropuertos_histo['fecha_fin'] = ''
df_aeropuertos_histo['A'] = ''
df_aeropuertos_histo.head(1)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,pbmo,Ano,fecha_ini,fecha_fin,A
0,BOG,EL DORADO,"Bogotá, D.C.","Bogotá, D.C.",Internacional,4.7017,-74.1469,1999-12-09,4E,Aerocivil,3800.0,60.0,8356.0,NaN,1999.0,1999-12-09,,


In [53]:
df_aeropuertos_histo = df_aeropuertos_histo.sort_values(by = ['fecha_ini'])
df_aeropuertos_histo.head(3)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,pbmo,Ano,fecha_ini,fecha_fin,A
249,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,25000.0,1957.0,1957-01-30,,
265,PSO,ANTONIO NARIÑO,Pasto,Nariño,Nacional,1.3964,-77.2910,1957-04-24,4C,Aerocivil,2177.0,40.0,5896.0,NaN,1957.0,1957-04-24,,
214,ATC,ATACO,Ataco,Tolima,Aeródromo,3.5837,-75.3891,1957-07-23,2A,Público,800.0,15.0,1519.0,4000.0,1957.0,1957-07-23,,


In [54]:
# Obtenemos la llave subrogada para el DF aeropuerto con historia
df_aeropuertos_histo.index.unique()
df_aeropuertos_histo.index = list(range(0,df_aeropuertos_histo.shape[0]))
df_aeropuertos_histo.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024],
           dtype='int64', length=1025)

In [55]:
siglas = df_aeropuertos_histo['sigla'].unique().tolist()
len(siglas)

298

In [56]:
df_aeropuertos_histo['Ano'] = df_aeropuertos_histo['Ano'].astype('int')

In [57]:
for i in siglas:
    filtro = df_aeropuertos_histo.sigla == i #True o flase
    ind = df_aeropuertos_histo[filtro].index.tolist() #lista de los indices donde es TRUE
    ind.sort() #Odenar los indices
    
    #print(i)
    #print(ind)
    
    #ESCRIBIMOS YES EN ES ESTADO ACTUAL Y ASIGNAMOS EN FECHA FIN UNA FECHA FUTURA
    df_aeropuertos_histo.loc[ind[-1], 'A'] = 'Y'
    df_aeropuertos_histo.loc[ind[-1], 'fecha_fin'] = '2100-01-01'
    
    for j in range(0,len(ind)-1):

        df_aeropuertos_histo.loc[ind[j], 'A'] = 'N'
        df_aeropuertos_histo.loc[ind[j], 'fecha_fin'] = (df_aeropuertos_histo.loc[ind[j+1], 'Ano']-1).astype(str) + '-12-31'  #**

In [58]:
df_aeropuertos_histo.columns

Index(['sigla', 'nombre', 'municipio', 'departamento', 'categoria', 'latitud',
       'longitud', 'fecha_construccion', 'clase', 'tipo', 'longitud_pista',
       'ancho_pista', 'elevacion', 'pbmo', 'Ano', 'fecha_ini', 'fecha_fin',
       'A'],
      dtype='object')

In [59]:
#DIMENSIÓN TIPO VUELO

df_tipo_vuelo = pd.DataFrame(df_vuelos['tipo_vuelo'].drop_duplicates())
df_tipo_vuelo['IDtipo_vuelo'] = list(range(0,df_tipo_vuelo.shape[0]))

df_tipo_vuelo

,tipo_vuelo,IDtipo_vuelo
0,T,0
114,R,1
135,A,2
214,C,3


In [60]:
#DIMENSIÓN TRÁFICO

df_trafico = pd.DataFrame(df_vuelos['trafico'].drop_duplicates())
df_trafico['IDtrafico'] = list(range(0,df_trafico.shape[0]))

df_trafico

,trafico,IDtrafico
0,N,0


In [61]:
#AHORA CREAMOS LAS LLAVES EN AEROPUERTO_HISTORIA PARA RELACIONAR ESTA DIMENSION CON LA TABLA DE HECHOS
#Se crean 2 dimensiones aeropuerto historia

df_aeropuertos_histo['IDaeropuerto'] = list(range(0,df_aeropuertos_histo.shape[0]))
df_aeropuertos_histo.head(1)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,pbmo,Ano,fecha_ini,fecha_fin,A,IDaeropuerto
0,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,25000.0,1957,1957-01-30,2013-12-31,N,0


In [62]:
df_aeropuertos_histo_ori = df_aeropuertos_histo.copy() 

df_aeropuertos_histo_ori = df_aeropuertos_histo_ori.drop(53) #Se eliminan dos registros por fechas erroneas
df_aeropuertos_histo_ori = df_aeropuertos_histo_ori.drop(74)

#Convertimos las variables fechas en formato date
df_aeropuertos_histo_ori['fecha_fin'] = pd.to_datetime(df_aeropuertos_histo_ori['fecha_fin'], format ="%Y/%m/%d")
df_aeropuertos_histo_ori['fecha_ini'] = pd.to_datetime(df_aeropuertos_histo_ori['fecha_ini'], format ="%Y/%m/%d")
df_aeropuertos_histo_ori['fecha_construccion'] = pd.to_datetime(df_aeropuertos_histo_ori['fecha_construccion'], format ="%Y/%m/%d")

df_aeropuertos_histo_ori.drop(["pbmo"], axis = 'columns', inplace = True)

df_aeropuertos_histo_ori.head(1)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,Ano,fecha_ini,fecha_fin,A,IDaeropuerto
0,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,1957,1957-01-30,2013-12-31,N,0


In [63]:
df_aeropuertos_histo_des = df_aeropuertos_histo.copy() 

df_aeropuertos_histo_des = df_aeropuertos_histo_des.drop(53) #Se eliminan dos registros por fechas erroneas
df_aeropuertos_histo_des = df_aeropuertos_histo_des.drop(74)

#Convertimos las variables fechas en formato date
df_aeropuertos_histo_des['fecha_fin'] = pd.to_datetime(df_aeropuertos_histo_des['fecha_fin'], format ="%Y/%m/%d")
df_aeropuertos_histo_des['fecha_ini'] = pd.to_datetime(df_aeropuertos_histo_des['fecha_ini'], format ="%Y/%m/%d")
df_aeropuertos_histo_des['fecha_construccion'] = pd.to_datetime(df_aeropuertos_histo_des['fecha_construccion'], format ="%Y/%m/%d")

df_aeropuertos_histo_des.drop(["pbmo"], axis = 'columns', inplace = True)

df_aeropuertos_histo_des.head(1)


,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,Ano,fecha_ini,fecha_fin,A,IDaeropuerto
0,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,1957,1957-01-30,2013-12-31,N,0


In [64]:
df_vuelos['or'] = list(zip(df_vuelos.origen,df_vuelos.ano))
df_vuelos['de'] = list(zip(df_vuelos.destino,df_vuelos.ano))

In [65]:
df_or = pd.DataFrame(df_vuelos['or'].unique())
df_de = pd.DataFrame(df_vuelos['de'].unique())

In [66]:
#df_de.head(5)

In [67]:
for i in range(0, df_or.shape[0]):
    filtro_ini = df_aeropuertos_histo['fecha_ini'].str.split("-", expand = True)[0].astype(int) <= df_or.loc[i, 0][1]
    filtro_fin = df_aeropuertos_histo['fecha_fin'].str.split("-", expand = True)[0].astype(int) >= df_or.loc[i, 0][1]
    filtro_aer = df_aeropuertos_histo['sigla'] == df_or.loc[i, 0][0]
    
    filtro = filtro_ini & filtro_fin & filtro_aer
    
    try:        df_or.loc[i, 'Id_or'] = df_aeropuertos_histo.IDaeropuerto[filtro].tolist()[0]
    except:     '' 

In [68]:
print(df_or.head())

             0  Id_or
0  (ADZ, 2010)   14.0
1  (BAQ, 2010)    NaN
2  (BOG, 2010)   59.0
3  (BUN, 2010)    7.0
4  (CAQ, 2010)    NaN


In [69]:
for i in range(0, df_de.shape[0]):
    filtro_ini = df_aeropuertos_histo['fecha_ini'].str.split("-", expand = True)[0].astype(int) <= df_de.loc[i, 0][1]
    filtro_fin = df_aeropuertos_histo['fecha_fin'].str.split("-", expand = True)[0].astype(int) >= df_de.loc[i, 0][1]
    filtro_aer = df_aeropuertos_histo['sigla'] == df_de.loc[i, 0][0]
    
    filtro = filtro_ini & filtro_fin & filtro_aer
        
    try:        df_de.loc[i, 'Id_de'] = df_aeropuertos_histo.IDaeropuerto[filtro].tolist()[0]
    except:     ''

In [70]:
df_de.Id_de

0       59.0
1       47.0
2        NaN
3        NaN
4       26.0
       ...  
773    822.0
774    824.0
775    960.0
776    795.0
777    942.0
Name: Id_de, Length: 778, dtype: float64

In [71]:
df_vuelos = df_vuelos.merge(df_or.rename(columns = {0 : 'or'}), how = 'left', on = "or" )

In [72]:
df_vuelos

,ano,mes,origen,destino,tipo_vuelo,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,trafico,periodo,or,de,Id_or
0,2010,3,ADZ,BOG,T,1.0,0.0,0.0,7.0,0.0,N,"(2010, 3)","(ADZ, 2010)","(BOG, 2010)",14.0
1,2010,5,ADZ,BOG,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 5)","(ADZ, 2010)","(BOG, 2010)",14.0
2,2010,6,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 6)","(ADZ, 2010)","(CLO, 2010)",14.0
3,2010,8,ADZ,CLO,T,1.0,0.0,0.0,6.0,0.0,N,"(2010, 8)","(ADZ, 2010)","(CLO, 2010)",14.0
4,2010,10,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 10)","(ADZ, 2010)","(CLO, 2010)",14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82381,2016,12,MVP,VVC,R,1.0,19.0,1400.0,11.0,400.0,N,"(2016, 12)","(MVP, 2016)","(VVC, 2016)",814.0
82382,2016,12,MVP,VVC,R,7.0,133.0,23800.0,100.0,1600.0,N,"(2016, 12)","(MVP, 2016)","(VVC, 2016)",814.0
82383,2016,12,VVC,EJA,R,1.0,0.0,2150.0,0.0,2150.0,N,"(2016, 12)","(VVC, 2016)","(EJA, 2016)",794.0
82384,2016,12,VVC,MVP,R,1.0,10.0,1640.0,10.0,1640.0,N,"(2016, 12)","(VVC, 2016)","(MVP, 2016)",794.0


In [73]:
df_vuelos = df_vuelos.merge(df_de.rename(columns = {0 : 'de'}), how = 'left', on = "de" )

In [74]:
df_vuelos

,ano,mes,origen,destino,tipo_vuelo,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,trafico,periodo,or,de,Id_or,Id_de
0,2010,3,ADZ,BOG,T,1.0,0.0,0.0,7.0,0.0,N,"(2010, 3)","(ADZ, 2010)","(BOG, 2010)",14.0,59.0
1,2010,5,ADZ,BOG,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 5)","(ADZ, 2010)","(BOG, 2010)",14.0,59.0
2,2010,6,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 6)","(ADZ, 2010)","(CLO, 2010)",14.0,47.0
3,2010,8,ADZ,CLO,T,1.0,0.0,0.0,6.0,0.0,N,"(2010, 8)","(ADZ, 2010)","(CLO, 2010)",14.0,47.0
4,2010,10,ADZ,CLO,T,1.0,0.0,0.0,8.0,0.0,N,"(2010, 10)","(ADZ, 2010)","(CLO, 2010)",14.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82381,2016,12,MVP,VVC,R,1.0,19.0,1400.0,11.0,400.0,N,"(2016, 12)","(MVP, 2016)","(VVC, 2016)",814.0,794.0
82382,2016,12,MVP,VVC,R,7.0,133.0,23800.0,100.0,1600.0,N,"(2016, 12)","(MVP, 2016)","(VVC, 2016)",814.0,794.0
82383,2016,12,VVC,EJA,R,1.0,0.0,2150.0,0.0,2150.0,N,"(2016, 12)","(VVC, 2016)","(EJA, 2016)",794.0,735.0
82384,2016,12,VVC,MVP,R,1.0,10.0,1640.0,10.0,1640.0,N,"(2016, 12)","(VVC, 2016)","(MVP, 2016)",794.0,814.0


In [75]:
# Se asigna el 'id_periodo' a la tabla de hechos 'df_vuelos'

In [76]:
#Asignamos periodo a la tabla de hechos
df_vuelos['periodo'] = list(zip(df_vuelos.ano, df_vuelos.mes))

In [77]:
#Realizamos Join entre las dos tablas "periodo"
df_vuelos = df_vuelos.merge(df_periodos.rename(columns = {'Periodo' : 'periodo'}), how = 'left', on = 'periodo')[['Id_Periodo', 'origen', 'destino', 'tipo_vuelo', 'vuelos','sillas', 'carga_ofrecida', 'pasajeros', 'carga_bordo','trafico','Id_or','Id_de']]

In [78]:
#Realizamos Join entre la dimension "tipo vuelo" y la tabla de hechos
df_vuelos = df_vuelos.merge(df_tipo_vuelo, how = 'left', on = 'tipo_vuelo')[['Id_Periodo', 'origen', 'destino', 'IDtipo_vuelo', 'vuelos','sillas', 'carga_ofrecida', 'pasajeros', 'carga_bordo','trafico','Id_or','Id_de']]

In [79]:
#Realizamos Join entre la dimension "trafico" y la tabla de hechos
df_vuelos = df_vuelos.merge(df_trafico, how = 'left', on = 'trafico')[['Id_Periodo', 'origen', 'destino', 'IDtipo_vuelo', 'vuelos','sillas', 'carga_ofrecida', 'pasajeros', 'carga_bordo','IDtrafico','Id_or','Id_de']]

In [80]:
#df_tipo_vuelo[df_tipo_vuelo['IDtipo_vuelo'] == 1]
#Seleccionamos las variables de interes en la tabla de hechos

df_vuelos = df_vuelos.rename(columns = {'Id_or': 'idaeropuertoHisOrigen','Id_de': 'idaeropuertoHisDestino'})[['Id_Periodo','idaeropuertoHisOrigen','idaeropuertoHisDestino', 'vuelos','sillas', 'carga_ofrecida', 'pasajeros', 'carga_bordo', 'IDtipo_vuelo','IDtrafico']]

In [81]:
df_vuelos

,Id_Periodo,idaeropuertoHisOrigen,idaeropuertoHisDestino,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,IDtipo_vuelo,IDtrafico
0,2,14.0,59.0,1.0,0.0,0.0,7.0,0.0,0,0
1,4,14.0,59.0,1.0,0.0,0.0,8.0,0.0,0,0
2,5,14.0,47.0,1.0,0.0,0.0,8.0,0.0,0,0
3,7,14.0,47.0,1.0,0.0,0.0,6.0,0.0,0,0
4,9,14.0,47.0,1.0,0.0,0.0,8.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
82381,83,814.0,794.0,1.0,19.0,1400.0,11.0,400.0,1,0
82382,83,814.0,794.0,7.0,133.0,23800.0,100.0,1600.0,1,0
82383,83,794.0,735.0,1.0,0.0,2150.0,0.0,2150.0,1,0
82384,83,794.0,814.0,1.0,10.0,1640.0,10.0,1640.0,1,0


In [82]:
# Se encuentra que datos de vuelos tienen una fecha menor a la fecha de construcción del aeropuerto de destino u origen
# se eliminan dichas incosistencias

In [83]:
df_vuelos.dropna(how = 'any',  subset = ['idaeropuertoHisOrigen', 'idaeropuertoHisDestino'])

,Id_Periodo,idaeropuertoHisOrigen,idaeropuertoHisDestino,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,IDtipo_vuelo,IDtrafico
0,2,14.0,59.0,1.0,0.0,0.0,7.0,0.0,0,0
1,4,14.0,59.0,1.0,0.0,0.0,8.0,0.0,0,0
2,5,14.0,47.0,1.0,0.0,0.0,8.0,0.0,0,0
3,7,14.0,47.0,1.0,0.0,0.0,6.0,0.0,0,0
4,9,14.0,47.0,1.0,0.0,0.0,8.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
82381,83,814.0,794.0,1.0,19.0,1400.0,11.0,400.0,1,0
82382,83,814.0,794.0,7.0,133.0,23800.0,100.0,1600.0,1,0
82383,83,794.0,735.0,1.0,0.0,2150.0,0.0,2150.0,1,0
82384,83,794.0,814.0,1.0,10.0,1640.0,10.0,1640.0,1,0


### IMPRESIÓN

In [84]:
#FECHA
df_periodos.head(2)

,Id_Periodo,Periodo,Ano,Mes,Nombre_Mes,Q,Dias
0,0,"(2010, 1)",2010,1,Enero,Q1,31
1,1,"(2010, 2)",2010,2,Febrero,Q1,28


In [85]:
#TIPO VUELO
df_tipo_vuelo.head(2)

,tipo_vuelo,IDtipo_vuelo
0,T,0
114,R,1


In [86]:
#TRAFICO
df_trafico.head(2)

,trafico,IDtrafico
0,N,0


In [87]:
#AEROPUERTO HISTORIA ORIGEN
df_aeropuertos_histo_ori.head(2)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,Ano,fecha_ini,fecha_fin,A,IDaeropuerto
0,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,1957,1957-01-30,2013-12-31,N,0
1,PSO,ANTONIO NARIÑO,Pasto,Nariño,Nacional,1.3964,-77.2910,1957-04-24,4C,Aerocivil,2177.0,40.0,5896.0,1957,1957-04-24,2013-12-31,N,1


In [88]:
#AEROPUERTO HISTORIA DESTINO
df_aeropuertos_histo_des.head(2)

,sigla,nombre,municipio,departamento,categoria,latitud,longitud,fecha_construccion,clase,tipo,longitud_pista,ancho_pista,elevacion,Ano,fecha_ini,fecha_fin,A,IDaeropuerto
0,MFS,MIRAFLORES,Miraflores,Guaviare,Aeródromo,1.3377,-71.9507,1957-01-30,3B,Público,1320.0,25.0,730.0,1957,1957-01-30,2013-12-31,N,0
1,PSO,ANTONIO NARIÑO,Pasto,Nariño,Nacional,1.3964,-77.2910,1957-04-24,4C,Aerocivil,2177.0,40.0,5896.0,1957,1957-04-24,2013-12-31,N,1


In [89]:
#TABLA DE HECHOS: VUELOS
df_vuelos.head(2)
df_vuelos.dtypes

Id_Periodo                  int64
idaeropuertoHisOrigen     float64
idaeropuertoHisDestino    float64
vuelos                    float64
sillas                    float64
carga_ofrecida            float64
pasajeros                 float64
carga_bordo               float64
IDtipo_vuelo                int64
IDtrafico                   int64
dtype: object

In [90]:

df_vuelos_nuevo = df_vuelos.dropna()

df_vuelos_nuevo_1 = df_vuelos_nuevo[df_vuelos_nuevo['idaeropuertoHisOrigen'].notna()]
df_vuelos_nuevo_2 = df_vuelos_nuevo_1[df_vuelos_nuevo_1['idaeropuertoHisOrigen'].isin(df_aeropuertos_histo_ori['IDaeropuerto'])]

df_vuelos_nuevo_3 = df_vuelos_nuevo_2[df_vuelos_nuevo_2['idaeropuertoHisDestino'].notna()]
df_vuelos_nuevo_4 = df_vuelos_nuevo_3[df_vuelos_nuevo_3['idaeropuertoHisDestino'].isin(df_aeropuertos_histo_des['IDaeropuerto'])]
df_vuelos_nuevo_5 = df_vuelos_nuevo_4[df_vuelos_nuevo_4['carga_bordo'].notna()]

df_vuelos_nuevo_6 = df_vuelos_nuevo_5[df_vuelos_nuevo_5['idaeropuertoHisDestino'] != float(53.0)]
df_vuelos_nuevo_7 = df_vuelos_nuevo_6[df_vuelos_nuevo_6['idaeropuertoHisOrigen'] != float(53.0)]
df_vuelos_nuevo_8 = df_vuelos_nuevo_7.dropna()
print(df_vuelos_nuevo_8.shape)
print(df_vuelos.shape)
#df_vuelos_nuevo.to_csv('C:/Users/carlosalarcon/Downloads/df_vuelos_nuevo.csv')
df_vuelos_nuevo_8[df_vuelos_nuevo_8['idaeropuertoHisDestino']==53.0]


(70344, 10)
(82386, 10)


,Id_Periodo,idaeropuertoHisOrigen,idaeropuertoHisDestino,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,IDtipo_vuelo,IDtrafico


### 3. Load

In [91]:
#df_temporal.to_csv('C:/Users/crist/Desktop/temporal.csv')

In [92]:
# Importar librerias

import pandas as pd
import pyodbc
import numpy
from datetime import date
from datetime import datetime
from datetime import timedelta
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

In [94]:
# Crear conexión a BBDD SQL Server

# Previamente se ha creado en SQL Server la base de datos ModeladoETLUniandes
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=LAPTOP-NDJ3I5A9;"
            "Database=ModeladoETLUniandes;"
            "UID=sa;"
            "PWD=camilo1122;"
           )

cnxn = pyodbc.connect(cnxn_str)

In [95]:
# Insertar datos en las dimensiones
# FECHA

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM Fecha")

cursor = cnxn.cursor()
df_periodos = df_periodos.astype(str)

for index, row in df_periodos.iterrows():
     cursor.execute("INSERT INTO Fecha values(?,?,?,?,?,?,?)", row.Id_Periodo, row.Periodo, row.Mes, row.Ano, row.Nombre_Mes, row.Q, row.Dias)
cnxn.commit()
cursor.close()

In [96]:
# TIPO VUELO

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM TipoVuelo")

cursor = cnxn.cursor()
df_tipo_vuelo = df_tipo_vuelo.astype(str)

for index, row in df_tipo_vuelo.iterrows():
     cursor.execute("INSERT INTO TipoVuelo values(?,?)", row.IDtipo_vuelo, row.tipo_vuelo)
cnxn.commit()
cursor.close()

In [97]:
# TIPO TRAFICO

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM TipoTrafico")

cursor = cnxn.cursor()
df_trafico = df_trafico.astype(str)

for index, row in df_trafico.iterrows():
     cursor.execute("INSERT INTO TipoTrafico values(?,?)", row.IDtrafico, row.trafico,)
cnxn.commit()
cursor.close()

In [98]:
# AEROPUERTO HISTORIA ORIGEN

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM AeropuertoHistoriaOrigen")

cursor = cnxn.cursor()

for index, row in df_aeropuertos_histo_ori.iterrows():
     cursor.execute("INSERT INTO AeropuertoHistoriaOrigen values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.IDaeropuerto, row.nombre, row.categoria, row.latitud, row.longitud, row.longitud_pista, row.ancho_pista, row.elevacion, row.fecha_construccion, row.clase, row.tipo, row.departamento, row.municipio, row.Ano, row.fecha_ini, row.fecha_fin, row.A, row.sigla)
cnxn.commit()
cursor.close()

In [99]:
# AEROPUERTO HISTORIA DESTINO

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM AeropuertoHistoriaDestino")

cursor = cnxn.cursor()

for index, row in df_aeropuertos_histo_des.iterrows():
     cursor.execute("INSERT INTO AeropuertoHistoriaDestino values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.IDaeropuerto, row.nombre, row.categoria, row.latitud, row.longitud, row.longitud_pista, row.ancho_pista, row.elevacion, row.fecha_construccion, row.clase, row.tipo, row.departamento, row.municipio, row.Ano, row.fecha_ini, row.fecha_fin, row.A, row.sigla)
cnxn.commit()
cursor.close()

In [100]:
# TABLA DE HECHOS: VUELOS

#Borrar data de la tabla
cursor = cnxn.cursor()
cursor.execute("DELETE FROM HechoVuelos")

cursor = cnxn.cursor()


for index, row in df_vuelos_nuevo_8.iterrows():
    cursor.execute("INSERT INTO HechoVuelos values(?,?,?,?,?,?,?,?,?,?)", row.vuelos, row.pasajeros, row.sillas, row.carga_ofrecida, row.carga_bordo, row. Id_Periodo, row.idaeropuertoHisOrigen, row.idaeropuertoHisDestino, row.IDtipo_vuelo, row.IDtrafico)
cnxn.commit()
cursor.close()


In [101]:
cnxn.close()

### ESTADISTICAS DESCRIPTIVAS DE LAS DIMENSIONES Y TABLA DE HECHOS

In [123]:
# Importación de librerías

import pandas as pd
import seaborn as sns
import missingno as msno ###
import numpy as np

In [124]:
df_aeropuertos_histo_des.describe(include = np.number).round(2)

,latitud,longitud,longitud_pista,ancho_pista,elevacion,Ano,IDaeropuerto
count,1023.00,1023.00,1023.00,1023.00,1023.00,1023.00,1023.00
mean,5.64,-73.14,1183.46,22.28,1010.47,2012.41,512.88
std,2.39,2.26,1441.90,24.38,1490.97,9.47,295.66
min,-1.75,-81.71,0.00,0.00,0.00,1957.00,0.00
25%,4.34,-74.85,249.00,5.00,358.50,2014.00,257.50
50%,5.29,-72.87,764.00,15.00,568.00,2015.00,513.00
75%,6.91,-71.61,1500.00,30.00,803.00,2016.00,768.50
max,13.36,-67.08,12279.00,163.00,9740.00,2016.00,1024.00


In [126]:
df_vuelos_nuevo_8.describe(include = np.number).round(2)

,Id_Periodo,idaeropuertoHisOrigen,idaeropuertoHisDestino,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,IDtipo_vuelo,IDtrafico
count,70344.00,70344.00,70344.00,70344.00,70344.00,70344.00,70344.00,70344.00,70344.00,70344.0
mean,44.37,282.57,282.63,14.52,1293.29,59954.88,1040.02,6213.80,0.44,0.0
std,23.15,304.04,304.14,37.70,5012.22,275193.98,4045.56,26362.92,0.58,0.0
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
25%,26.00,45.00,45.00,1.00,0.00,0.00,2.00,0.00,0.00,0.0
50%,46.00,88.00,88.00,2.00,0.00,0.00,8.00,90.00,0.00,0.0
75%,64.00,468.00,467.00,10.00,294.00,10647.25,225.00,1138.00,1.00,0.0
max,83.00,1019.00,1019.00,2004.00,81019.00,4758000.00,65195.00,467438.00,3.00,0.0


In [128]:
df_tipo_vuelo.describe(exclude = np.number).round(2)

,tipo_vuelo,IDtipo_vuelo
count,4,4
unique,4,4
top,C,1
freq,1,1


In [129]:
df_trafico.describe(exclude = np.number).round(2)

,trafico,IDtrafico
count,1,1
unique,1,1
top,N,0
freq,1,1


In [132]:
df_periodos.describe(exclude = np.number).round(2)

,Id_Periodo,Periodo,Ano,Mes,Nombre_Mes,Q,Dias
count,84,84,84,84,84,84,84
unique,84,84,7,12,12,4,3
top,76,"(2014, 4)",2016,4,Abril,Q3,31
freq,1,1,12,7,7,21,49
